In [10]:
from decouple import config
from together import Together
import requests
import time
import base64
from datetime import datetime, timedelta
# Configuración
TOGETHER_API_KEY = config('TOGETHER_API_KEY')
ESP32_IP = "192.168.249.200"
CAPTURE_URL = f"http://{ESP32_IP}/capture"
COMMAND_URL = f"http://{ESP32_IP}/cmd"
INTERVAL = 1  # Intervalo en segundos entre capturas

# Inicializar cliente Together
client = Together()

# Variables para la rutina de movimientos
step_counter = 0
total_steps = 28  # Total de pasos en la rutina

def get_routine_movement(step):
    """Determina el movimiento basado en el paso actual de la rutina"""
    if step < 5:  # Primeros 5 pasos
        return "up"
    elif step == 5:
        return "right"
    elif step > 5 and step < 11:  # Siguientes 5 pasos
        return "up"
    elif step == 11:
        return "left"
    elif step > 11 and step < 17:  # Siguientes 5 pasos
        return "up"
    elif step == 17:
        return "left"
    elif step > 17 and step < 23:  # Siguientes 5 pasos
        return "up"
    elif step >= 23 and step < 28:  # Últimos 5 pasos
        return "down"
    else:
        return "stop"

def capture_image_from_esp32():
    """Captura la imagen del ESP32-CAM y la convierte a base64"""
    try:
        response = requests.get(CAPTURE_URL, timeout=5)
        if response.status_code == 200:
            image_base64 = base64.b64encode(response.content).decode('utf-8')
            return f"data:image/jpeg;base64,{image_base64}"
        else:
            print(f"Error capturando imagen: {response.status_code}")
            return None
    except Exception as e:
        print(f"Error de conexión: {e}")
        return None

def send_command_to_esp32(command):
    """Envía el comando al ESP32"""
    try:
        response = requests.get(f"{COMMAND_URL}?cmd={command}", timeout=5)
        if response.status_code == 200:
            print(f"Comando enviado: {command}")
        else:
            print(f"Error enviando comando: {response.status_code}")
    except Exception as e:
        print(f"Error de conexión: {e}")

def get_direction(image_url): 
    """Obtiene la dirección usando el modelo de IA y limpia la salida."""
    prompt = """Analyze this image and determine the direction of the signal shown in it. 
    The possible answers are: 'up', 'down', 'left', 'right', 'stop', or 'no signal'. 
    If you are unsure or the image does not contain a signal, return only 'no signal'. 
    Provide only the exact answer as a single word: 'up', 'down', 'left', 'right', 'stop', or 'no signal'. 
    Do not include any explanations or additional text."""
    
    # Etiquetas esperadas
    valid_labels = {'up', 'down', 'left', 'right', 'stop', 'no signal'}

    try:
        response = client.chat.completions.create(
            model="meta-llama/Llama-3.2-11B-Vision-Instruct-Turbo",
            messages=[
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": prompt},
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": image_url,
                            },
                        },
                    ],
                }
            ]
        )
        # Limpia la respuesta del modelo
        raw_output = response.choices[0].message.content.strip().lower()  # Convertir a minúsculas y quitar espacios
        cleaned_output = raw_output.replace('"', '').replace("'", '')  # Quitar comillas si existen
        return cleaned_output if cleaned_output in valid_labels else "no signal"  # Validar contra etiquetas esperadas
    except Exception as e:
        print(f"Error en el procesamiento de IA: {e}")
        return "no signal"


def main():
    print("Iniciando sistema de control...")
    print(f"ESP32 IP: {ESP32_IP}")
    
    next_capture_time = datetime.now()
    global step_counter
    
    while True:
        current_time = datetime.now()
        
        # Solo ejecutar si ha pasado el intervalo de tiempo
        if current_time >= next_capture_time:
            print(f"\nCapturando imagen a las: {current_time.strftime('%H:%M:%S')}")
            
            # Capturar imagen
            image_url = capture_image_from_esp32()
            if image_url:
                print("Imagen capturada exitosamente")
                
                # Obtener el siguiente movimiento de la rutina
                movement = get_routine_movement(step_counter)
                print(f"Paso {step_counter + 1}: Enviando comando '{movement}'")
                send_command_to_esp32(movement)
                
                # Incrementar contador de pasos
                step_counter += 1
                
                # Si llegamos al final de la rutina, mantener en "stop"
                if step_counter > total_steps:
                    print("Rutina completada")
                    send_command_to_esp32("stop")
                    break
            
            # Calcular el siguiente tiempo de captura
            next_capture_time = current_time + timedelta(seconds=INTERVAL)
        
        # Pequeña pausa para no saturar el CPU
        time.sleep(0.1)

if __name__ == "__main__":
    main()

Iniciando sistema de control...
ESP32 IP: 192.168.249.200

Capturando imagen a las: 12:11:20
Imagen capturada exitosamente
Paso 1: Enviando comando 'up'
Comando enviado: up

Capturando imagen a las: 12:11:22
Error de conexión: HTTPConnectionPool(host='192.168.249.200', port=80): Max retries exceeded with url: /capture (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x77505d192e30>, 'Connection to 192.168.249.200 timed out. (connect timeout=5)'))

Capturando imagen a las: 12:11:27
Imagen capturada exitosamente
Paso 2: Enviando comando 'up'
Comando enviado: up

Capturando imagen a las: 12:11:30
Imagen capturada exitosamente
Paso 3: Enviando comando 'up'
Comando enviado: up

Capturando imagen a las: 12:11:31
Imagen capturada exitosamente
Paso 4: Enviando comando 'up'
Comando enviado: up

Capturando imagen a las: 12:11:33
Imagen capturada exitosamente
Paso 5: Enviando comando 'up'
Comando enviado: up

Capturando imagen a las: 12:11:34
Imagen capturada exitosamen